In [ ]:
import pandas as pd
import numpy as np

# Importar módulos necesarios para la construcción y evaluación del modelo
from sklearn.model_selection import GroupKFold, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor # HistGradientBoostingRegressor está comentado
from sklearn.base import BaseEstimator, TransformerMixin


# Transformador personalizado para la codificación de destino suavizada (Smoothed Target Encoding)
class SmoothedTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None, smoothing=10.0):
        self.cols = cols
        self.smoothing = float(smoothing)
        self.maps_ = {} # Diccionario para almacenar mapeos
        self.global_mean_ = None # Media global de la variable objetivo

    def fit(self, X, y):
        # Ajustar el codificador a los datos de entrenamiento
        X = pd.DataFrame(X, copy=True)
        y = pd.Series(y)
        self.global_mean_ = y.mean() # Calcular la media global
        tmp = X.copy()
        tmp["_y"] = y.values
        self.maps_.clear()
        if self.cols:
            for c in self.cols:
                if c not in tmp.columns:
                    continue
                # Calcular la media del destino suavizada para cada categoría
                g = tmp.groupby(c)["_y"].agg(["mean","count"]).rename(columns={"mean":"mu","count":"N"})
                enc = (g["N"]*g["mu"] + self.smoothing*self.global_mean_) / (g["N"] + self.smoothing)
                self.maps_[c] = enc.to_dict() # Almacenar mapeo
        return self

    def transform(self, X):
        # Transformar los datos usando los mapeos aprendidos
        X = pd.DataFrame(X, copy=True)
        for c, mp in self.maps_.items():
            if c in X.columns:
                # Aplicar mapeo y llenar valores faltantes con la media global
                X[c] = X[c].map(mp).fillna(self.global_mean_)
        return X

    def get_feature_names_out(self, input_features=None):
        # Obtener nombres de las características de salida
        return np.array(input_features if input_features is not None else (self.cols or []))


# Función de ayuda para encontrar la primera columna entre una lista de opciones
def first_col(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

# Función de ayuda para convertir una serie a una proporción entre 0 y 1
def to_ratio_0_1(s):
    x = pd.to_numeric(s, errors="coerce").astype(float)
    if np.nanmean(x > 1) > 0.1:
        x = x / 100.0
    return x.clip(0, 1) # Recortar valores para que estén entre [0, 1]

# Función de ayuda para asignar un estado de "semáforo" basado en un valor
def semaforo(v):
    if v >= 0.85: return "Verde"
    if v >= 0.60: return "Amarillo"
    return "Rojo"


# Función para cargar y preprocesar los datos para entrenamiento
def load_and_preprocess_data(file_path):
    df = pd.read_excel(file_path, sheet_name="Dataset de Testeo")

    # Identificar la columna para el ID de la obra
    OBRA_ID_COL = first_col(df, ["Obra","PEP"])
    if OBRA_ID_COL is None:
        # Crear OBRA_ID si no se encuentra
        df["OBRA_ID"] = df[first_col(df, df.columns)].astype(str).str.extract(r"(SCCOM\d+|[A-Z]{2,}\d+)", expand=False)
        OBRA_ID_COL = "OBRA_ID"

    # Identificar y procesar la columna de fecha de inicio de auditoría
    fecha_ini_col = first_col(df, ["Fecha Inicio Auditoría","Fecha Inicio Auditoria","Fecha Auditoría","Fecha Auditoria"])
    if fecha_ini_col is not None:
        df[fecha_ini_col] = pd.to_datetime(df[fecha_ini_col], dayfirst=True, errors="coerce")
        # Ordenar por ID de obra y fecha
        df = df.sort_values([OBRA_ID_COL, fecha_ini_col])
    else:
        df = df.sort_values([OBRA_ID_COL]).reset_index(drop=True)

    # Renombrar columnas para consistencia
    rename_map = {
        "Administrador": "Administrador de Obra",
        "Oficina Técnica": "Oficina Tecnica",
        "Jefe de bodega": "Jefes de Bodega",
        "Avance Físico": "Avance Fisico",
    }
    for k,v in rename_map.items():
        if k in df.columns and v not in df.columns:
            df[v] = df[k]

    # Convertir columnas de evaluación y stock a numéricas
    for c in ["Evaluación Obra","Evaluación Bodega","Evaluación OT","Evaluación AO","Stock","Avance Fisico"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Identificar y procesar la columna de porcentaje de inventario
    inv_pct_col = first_col(df, [
        "Valor porcentual sobre stock", "% Diferencia sobre Stock",
        "Porcentaje sobre Stock", "InvPctSobreStock", "Diferencia Absoluta %"
    ])
    if inv_pct_col is None:
        df["InvPctSobreStock_EXCEL"] = np.nan
        inv_pct_col = "InvPctSobreStock_EXCEL"
    df[inv_pct_col] = to_ratio_0_1(df[inv_pct_col])

    # === AÑADIDO: Resultado de la última evaluación de Obra ===
    # Añadir columnas para los resultados de la última evaluación para diferentes áreas
    df["Resultados Ultima Evaluacion Obra"] = df.groupby(OBRA_ID_COL)["Evaluación Obra"].shift(1)

    df["Resultados Ultima Bodega"] = df.groupby(OBRA_ID_COL)["Evaluación Bodega"].shift(1) if "Evaluación Bodega" in df.columns else np.nan
    ot_src = first_col(df, ["Evaluación OT","Evaluación OTDiferencia Absoluta"])
    df["Resultados Ultima Ev OT"] = df.groupby(OBRA_ID_COL)[ot_src].shift(1) if ot_src else np.nan
    df["Resultados Ultima AO"] = df.groupby(OBRA_ID_COL)["Evaluación AO"].shift(1) if "Evaluación AO" in df.columns else np.nan
    df["Resultados Ultima auditoria Inventario"] = df.groupby(OBRA_ID_COL)[inv_pct_col].shift(1)

    # Añadir una columna para el número de auditorías previas
    if "Cantidad de Auditorias" not in df.columns:
        df["Cantidad de Auditorias"] = df.groupby(OBRA_ID_COL).cumcount()

    # Imputar valores faltantes inicialmente por Empresa
    for c in ["Resultados Ultima Bodega", "Resultados Ultima Ev OT", "Resultados Ultima AO",
              "Resultados Ultima auditoria Inventario", "Avance Fisico", "Stock",
              "Resultados Ultima Evaluacion Obra"]: # <- Añadido
        if c in df.columns and "Empresa" in df.columns:
            df[c] = df.groupby("Empresa")[c].transform(lambda s: s.fillna(s.mean()))
            df[c] = df[c].fillna(df[c].mean()) # Llenar los NaNs restantes con la media global

    return df, OBRA_ID_COL


# Función para hacer predicciones en nuevos datos
def predecir(df_nuevo: pd.DataFrame, pipe, cols_present, thr_recall70, best_f1_row, usar_umbral_recall70: bool=True) -> pd.DataFrame:
    """
    Puntúa nuevas filas con el regresor; aplica semáforo y alarma.
    Scores new rows with the regressor; applies semaphore and alarm.
    """
    d = df_nuevo.copy()

    # Renombrar columnas en los nuevos datos para consistencia
    rmap = {
        "Administrador": "Administrador de Obra",
        "Oficina Técnica": "Oficina Tecnica",
        "Jefe de bodega": "Jefes de Bodega",
        "Avance Físico": "Avance Fisico",
    }
    for k,v in rmap.items():
        if k in d.columns and v not in d.columns:
            d[v] = d[k]

    # Añadir columnas faltantes con valores NaN
    for c in cols_present:
        if c not in d.columns:
            d[c] = np.nan

    # Llenar valores faltantes en columnas categóricas y de roles con 'missing_val'
    cat_ohe_cols = ["Empresa", "Auditor"]
    roles_te_cols = ["Gerente de Proyecto", "Administrador de Obra", "Oficina Tecnica", "Jefes de Bodega"]
    for c in cat_ohe_cols + roles_te_cols:
        if c in d.columns:
            d[c] = d[c].fillna('missing_val').astype(str)

    # Hacer predicciones usando el pipeline entrenado
    y_hat = pipe.predict(d[cols_present])

    # Determinar el umbral a usar para la clasificación
    thr = float(thr_recall70) if usar_umbral_recall70 else float(best_f1_row.threshold)

    # Crear el DataFrame de salida con predicciones, estado del semáforo y alarma
    out = pd.DataFrame({
        "pred_eval_obra": y_hat,
        "semaforo": [semaforo(v) for v in y_hat],
        "alarma_<70%": (y_hat < thr).astype(int),
        "umbral_usado": thr
    })
    return out

# Función principal para ejecutar el proceso de entrenamiento y predicción del modelo
def main():
    # Cargar y preprocesar los datos de entrenamiento
    df, OBRA_ID_COL = load_and_preprocess_data("Prueba prediccion Auditorias.xlsx")

    # Convertir la variable objetivo a numérica y filtrar valores inválidos
    df["Evaluación Obra"] = pd.to_numeric(df["Evaluación Obra"], errors="coerce")
    df = df[df["Evaluación Obra"].between(1e-3, 1-1e-3)].copy()

    # === Modificado: Se agrega "Resultados Ultima Evaluacion Obra" a USE_COLS ===
    # Definir las columnas a usar para el entrenamiento
    USE_COLS = [
        "Resultados Ultima Bodega",
        "Resultados Ultima Ev OT",
        "Resultados Ultima AO",
        "Resultados Ultima auditoria Inventario",
        "Gerente de Proyecto",
        "Administrador de Obra",
        "Oficina Tecnica",
        "Jefes de Bodega",
        "Empresa",
        "Avance Fisico",
        "Cantidad Inv Generales Previos",
        "Stock",
        "Cantidad de Auditorias",
        "Resultados Ultima Evaluacion Obra", # Añadido
    ]

    # Verificar columnas faltantes y añadirlas con valores NaN
    missing = [c for c in USE_COLS if c not in df.columns]
    if missing:
        print("¡Ojo! Faltaban columnas y se crearán vacías (se imputan):", missing)
        for c in missing:
            df[c] = np.nan

    # Definir columnas categóricas y de roles
    cat_ohe_cols = ["Empresa", "Auditor"]
    roles_te_cols = ["Gerente de Proyecto", "Administrador de Obra", "Oficina Tecnica", "Jefes de Bodega"]

    # Llenar valores faltantes en columnas categóricas y de roles
    for c in cat_ohe_cols + roles_te_cols:
        if c in df.columns:
            df[c] = df[c].fillna('missing_val').astype(str)

    # Obtener la lista de columnas presentes en el DataFrame
    cols_present = USE_COLS[:]
    y = df["Evaluación Obra"].values # Variable objetivo
    groups = df[OBRA_ID_COL].astype(str).values # Variable de agrupación para validación cruzada

    # Eliminar columnas con todos los valores NaN
    all_nan = [c for c in cols_present if df[c].isna().all()]
    if all_nan:
        print("Eliminar 100% NaN:", all_nan)
        cols_present = [c for c in cols_present if c not in all_nan]

    # Separar columnas por tipo para el preprocesamiento
    roles_te = [c for c in roles_te_cols if c in cols_present]
    cat_ohe  = [c for c in cat_ohe_cols if c in cols_present]
    num_cols = [c for c in cols_present if c not in roles_te + cat_ohe]

    # Eliminar 'Resultados Ultima Bodega' de num_cols para aplicar escalado separado
    bodega_col = "Resultados Ultima Bodega"
    if bodega_col in num_cols:
        num_cols.remove(bodega_col)
        bodega_col_present = True
    else:
        bodega_col_present = False


    print(f"Obras únicas: {pd.Series(groups).nunique()} | Filas: {len(df)}")
    print("Numéricas (excl. Bodega):", num_cols)
    print("Roles (TE):", roles_te)
    print("Categóricas OHE:", cat_ohe)
    if bodega_col_present:
        print("Resultados Ultima Bodega (MinMaxScaler):", [bodega_col])


    # Definir los pasos de preprocesamiento usando ColumnTransformer
    transformers = []
    if num_cols:
        transformers.append(("num", SimpleImputer(strategy="median"), num_cols)) # Imputar columnas numéricas con la mediana
    if roles_te:
        transformers.append(("roles_te", SmoothedTargetEncoder(cols=roles_te, smoothing=10), roles_te)) # Aplicar Smoothed Target Encoding a columnas de roles
    if cat_ohe:
        transformers.append(("ohe", OneHotEncoder(handle_unknown="infrequent_if_exist",
                                                  min_frequency=5, sparse_output=False), cat_ohe)) # Aplicar One-Hot Encoding a columnas categóricas
    if bodega_col_present:
         transformers.append(("bodega_scaler", Pipeline([('imputer', SimpleImputer(strategy="median")), ('scaler', MinMaxScaler())]), [bodega_col])) # Añadir MinMaxScaler para Bodega


    # Verificar si hay transformadores definidos
    if not transformers:
        raise ValueError("No hay columnas para entrenar el modelo.")

    # Crear el ColumnTransformer
    pre = ColumnTransformer(
        transformers=transformers,
        remainder="drop", # Eliminar columnas no especificadas en los transformadores
        verbose_feature_names_out=False
    )

    # Definir el modelo (Gradient Boosting Regressor)
    model = GradientBoostingRegressor( # Cambiado a GradientBoostingRegressor
        learning_rate=0.03,
        max_depth=5,
        n_estimators=100, # Añadido n_estimators para GradientBoostingRegressor
        random_state=42
    )

    # Crear el pipeline con preprocesamiento y modelo
    pipe = Pipeline([
        ("pre", pre),
        ("model", model)
    ])

    # Configurar GroupKFold para validación cruzada
    n_splits = max(2, min(5, pd.Series(groups).nunique()))
    gkf = GroupKFold(n_splits=n_splits)

    # Realizar validación cruzada e imprimir métricas
    r2 = cross_val_score(pipe, df[cols_present], y, cv=gkf, groups=groups, scoring="r2", n_jobs=-1).mean()
    mae = -cross_val_score(pipe, df[cols_present], y, cv=gkf, groups=groups, scoring="neg_mean_absolute_error", n_jobs=-1).mean()
    rmse = np.sqrt(-cross_val_score(pipe, df[cols_present], y, cv=gkf, groups=groups, scoring="neg_mean_squared_error", n_jobs=-1).mean())
    print(f"\nCV (GroupKFold) -> R²: {r2:.3f} | MAE: {mae:.3f} | RMSE: {rmse:.3f}")

    # Ajustar el pipeline a todos los datos de entrenamiento
    pipe.fit(df[cols_present], y)
    # Obtener predicciones fuera de pliegue para evaluación de clasificación
    y_pred_reg = cross_val_predict(pipe, df[cols_present], y, cv=gkf, groups=groups, n_jobs=-1)

    # Imprimir importancia de las características
    feature_importances = None
    try:
        # Verificar si el modelo tiene feature_importances_ o un atributo similar
        if hasattr(pipe.named_steps['model'], 'feature_importances_'):
            feature_importances = pipe.named_steps['model'].feature_importances_
            # Obtener nombres de las características después del preprocesamiento
            feature_names = pipe.named_steps['pre'].get_feature_names_out()
            # Crear un DataFrame para la importancia de las características
            importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
            importance_df = importance_df.sort_values('Importance', ascending=False)
            print("\nImportancia de las Características:")
            display(importance_df)
        else:
            print("\nNo se pudo obtener la importancia de las características: El modelo no tiene un atributo 'feature_importances_'.")
            print("Considere usar un modelo como RandomForestRegressor o GradientBoostingRegressor si la importancia de las características es crucial.")

    except Exception as e:
        print(f"\nOcurrió un error inesperado al intentar obtener la importancia de las características: {e}")


    # Evaluar el rendimiento de clasificación para diferentes umbrales
    y_true_alarm = (y < 0.70).astype(int) # Definir las etiquetas de alarma verdaderas
    grid = np.linspace(0.40, 0.80, 41) # Definir una cuadrícula de umbrales a evaluar

    rows = []
    for t in grid:
        y_hat = (y_pred_reg < t).astype(int) # Predecir alarma basada en el umbral
        # Calcular métricas de clasificación
        TP = ((y_true_alarm==1) & (y_hat==1)).sum()
        FP = ((y_true_alarm==0) & (y_hat==1)).sum()
        FN = ((y_true_alarm==1) & (y_hat==0)).sum()
        precision = TP / (TP+FP) if (TP+FP)>0 else 0.0
        recall    = TP / (TP+FN) if (TP+FN)>0 else 0.0
        f1        = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0
        rows.append((t, precision, recall, f1))

    # Crear un DataFrame con los resultados de la evaluación de umbrales
    thr_df = pd.DataFrame(rows, columns=["threshold","precision","recall","f1"])
    # Encontrar el umbral que maximiza el puntaje F1
    best_f1_row = thr_df.iloc[thr_df["f1"].idxmax()]
    print("\nUmbral (max F1):", round(best_f1_row.threshold,3),
          "| P:", round(best_f1_row.precision,2),
          "| R:", round(best_f1_row.recall,2),
          "| F1:", round(best_f1_row.f1,2))

    # Encontrar el umbral mínimo que alcanza un recall de al menos 0.70
    cand = thr_df[thr_df["recall"] >= 0.70]
    if not cand.empty:
        thr_recall70 = float(cand.threshold.min())
        print("Umbral mínimo con recall≥0.70:", round(thr_recall70,3))
    else:
        thr_recall70 = float(best_f1_row.threshold)
        print("No se alcanzó recall≥0.70; uso umbral de F1:", round(thr_recall70,3))

    # Establecer el umbral final para la clasificación de alarma
    THRESHOLD = float(thr_recall70)
    y_hat_final = (y_pred_reg < THRESHOLD).astype(int) # Predicciones finales de alarma
    print(f"\n=== Clasificación de alarma (<70%) con umbral={THRESHOLD:.3f} (OOF por obra) ===")
    # Imprimir el informe de clasificación
    print(classification_report(y_true_alarm, y_hat_final, target_names=["OK","AUDITAR"]))

    # Cargar el archivo de plantilla para nuevas predicciones
    try:
        df_plantilla = pd.read_excel("plantilla_pre_auditoria.xlsx")
    except FileNotFoundError:
        print("\nError: No se encontró el archivo 'plantilla_pre_auditoria.xlsx'.")
        return

    # Llenar valores faltantes en columnas categóricas y de roles en los datos de la plantilla
    cat_ohe_cols = ["Empresa", "Auditor"]
    roles_te_cols = ["Gerente de Proyecto", "Administrador de Obra", "Oficina Tecnica", "Jefes de Bodega"]
    for c in cat_ohe_cols + roles_te_cols:
        if c in df_plantilla.columns:
            df_plantilla[c] = df_plantilla[c].fillna('missing_val').astype(str)

    # Hacer predicciones en los datos de la plantilla
    resultados_prediccion = predecir(df_plantilla, pipe, cols_present, thr_recall70, best_f1_row, usar_umbral_recall70=True)

    # Concatenar los datos de la plantilla con los resultados de la predicción
    df_plantilla_con_prediccion = pd.concat([df_plantilla, resultados_prediccion], axis=1)

    # Guardar las predicciones y la importancia de las características en un archivo de Excel
    try:
        with pd.ExcelWriter("plantilla_con_predicciones.xlsx") as writer:
            df_plantilla_con_prediccion.to_excel(writer, sheet_name='Predicciones', index=False)
            if feature_importances is not None:
                 importance_df.to_excel(writer, sheet_name='Importancia de Variables', index=False)

        print("\nPredicciones y Importancia de Variables guardadas exitosamente en 'plantilla_con_predicciones.xlsx'.")
    except Exception as e:
        print(f"\nError al guardar el archivo: {e}")

# Ejecutar la función principal cuando se ejecuta el script
if __name__ == "__main__":
    main()

Eliminar 100% NaN: ['Resultados Ultima auditoria Inventario']
Obras únicas: 187 | Filas: 381
Numéricas (excl. Bodega): ['Resultados Ultima Ev OT', 'Resultados Ultima AO', 'Avance Fisico', 'Cantidad Inv Generales Previos', 'Stock', 'Cantidad de Auditorias', 'Resultados Ultima Evaluacion Obra']
Roles (TE): ['Gerente de Proyecto', 'Administrador de Obra', 'Oficina Tecnica', 'Jefes de Bodega']
Categóricas OHE: ['Empresa']
Resultados Ultima Bodega (MinMaxScaler): ['Resultados Ultima Bodega']

CV (GroupKFold) -> R²: -0.027 | MAE: 0.128 | RMSE: 0.167

Importancia de las Características:


,Feature,Importance
9,Oficina Tecnica,0.497421
10,Jefes de Bodega,0.186154
8,Administrador de Obra,0.086428
4,Stock,0.038124
7,Gerente de Proyecto,0.036723
2,Avance Fisico,0.032839
6,Resultados Ultima Evaluacion Obra,0.024402
3,Cantidad Inv Generales Previos,0.022609
23,Resultados Ultima Bodega,0.020449
1,Resultados Ultima AO,0.018346



Umbral (max F1): 0.77 | P: 0.54 | R: 0.95 | F1: 0.69
Umbral mínimo con recall≥0.70: 0.71

=== Clasificación de alarma (<70%) con umbral=0.710 (OOF por obra) ===
              precision    recall  f1-score   support

          OK       0.61      0.42      0.49       195
     AUDITAR       0.54      0.72      0.62       186

    accuracy                           0.56       381
   macro avg       0.57      0.57      0.56       381
weighted avg       0.58      0.56      0.55       381


Predicciones y Importancia de Variables guardadas exitosamente en 'plantilla_con_predicciones.xlsx'.
